In [ ]:
!sudo apt install python3.8-full python3-pip
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1
!python --version

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
import json
import numpy as np
import os, cv2, glob, json, gc
import pandas as pd
import itertools
from itertools import chain
from moviepy.editor import VideoFileClip
import skvideo.io
from tqdm import tqdm
import circstat as CS
import scipy as sc
import math

In [ ]:
np.float = np.float64
np.int = np.int_

# COCO limb sequence (0-based indexing)
coco_limb_sequence = [
    (0, 1),  # Nose to Left Eye
    (0, 2),  # Nose to Right Eye
    (1, 3),  # Left Eye to Left Ear
    (2, 4),  # Right Eye to Right Ear
    (5, 7),  # Left Shoulder to Left Elbow
    (7, 9),  # Left Elbow to Left Wrist
    (6, 8),  # Right Shoulder to Right Elbow
    (8, 10), # Right Elbow to Right Wrist
    (5, 6),  # Left Shoulder to Right Shoulder
    (5, 11), # Left Shoulder to Left Hip
    (6, 12), # Right Shoulder to Right Hip
    (11, 12),# Left Hip to Right Hip
    (11, 13),# Left Hip to Left Knee
    (13, 15),# Left Knee to Left Ankle
    (12, 14),# Right Hip to Right Knee
    (14, 16) # Right Knee to Right Ankle
]

limb_sequence = [
    (0,14),
    (0,15),
    (14,16),
    (15,17),
    (0,1),
    (1,2),
    (2,3),
    (3,4),
    (1,5),
    (5,6),
    (6,7),
    (1,8),
    (1,11),
    (8,9),
    (9,10),
    (11,12),
    (12,13),
    ]

mapping = {0:0,1:15,2:16,3:17,4:18,5:2,6:5,7:3,8:6,9:4,10:7,11:9,12:12,13:10,14:13,15:11,16:14}

# COCO part list
part_list = {
    0: "Nose",
    1: "Left Eye",
    2: "Right Eye",
    3: "Left Ear",
    4: "Right Ear",
    5: "Left Shoulder",
    6: "Right Shoulder",
    7: "Left Elbow",
    8: "Right Elbow",
    9: "Left Wrist",
    10: "Right Wrist",
    11: "Left Hip",
    12: "Right Hip",
    13: "Left Knee",
    14: "Right Knee",
    15: "Left Ankle",
    16: "Right Ankle"
}


colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0],
          [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255],
          [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85],[255, 0, 0]]

In [ ]:
def get_best_instance(instances):
    """
    Given a list of instances, return the index of the instance where keypoints are highest confidence.

    Parameters:
    instances (list): List of instances, where each instance contains a 'bbox' key with bounding box coordinates.

    Returns:
    int: The index of the instance whose center is closest to the frame center.
    """

    best_score = 0

    for e, instance in enumerate(instances):
        
        confidence = instance['keypoint_scores']

        if len(confidence) == 17:
            score = sum(confidence)

            if score > best_score:
                best_score = score
                n_instance = e

    
    return n_instance


def get_center_instance(instances, center_x, center_y):
    """
    Given a list of instances, return the index of the instance whose center is closest to the center of the video frame.

    Parameters:
    instances (list): List of instances, where each instance contains a 'bbox' key with bounding box coordinates.
    center_x (float): The x-coordinate of the frame center.
    center_y (float): The y-coordinate of the frame center.

    Returns:
    int: The index of the instance whose center is closest to the frame center.
    """

    distances = []
    for e, instance in enumerate(instances):
        
        bbox_x, bbox_y, bbox_width, bbox_height = instance['bbox'][0]
        bbox_center_x = bbox_x + bbox_width / 2
        bbox_center_y = bbox_y + bbox_height / 2

        # Calculate the Euclidean distance between the centers
        distance = math.sqrt((bbox_center_x - center_x)**2 + (bbox_center_y - center_y)**2)
        distances.append(distance)
    
    n_instance = distances.index(min(distances))
    
    return n_instance
        

def analyze_file(file_path, threshold=0.8):
    with open(file_path, 'r') as file:
        data = json.load(file)
    
    results = []
    for frame in data:
        frame_id = frame['frame_id']
        if frame['instances']:  # Ensure there is at least one detection
            instance_idx = get_best_instance(frame['instances'])
            first_instance = frame['instances'][instance_idx]
            keypoint_scores = first_instance['keypoint_scores']
            
            if len(keypoint_scores) != 17:
                continue
            
            all_above_thr = all(score > threshold for score in keypoint_scores)
            results.append({
                'frame_id': frame_id,
                'first_detection_keypoints': first_instance['keypoints'],
                'first_detection_confidence': keypoint_scores,
                'all_keypoints_above_thr': all_above_thr
            })
        else:
            continue
    
    return results


def find_continuous_good_blocks(analysis_results):
    good_blocks = []
    current_block = []
    
    for result in analysis_results:
        if result['all_keypoints_above_thr']:
            current_block.append(result)
        else:
            if len(current_block) >= 30:
                good_blocks.append(current_block)
            current_block = []
    
    # Check if the last block in the sequence is a good block
    if len(current_block) >= 30:
        good_blocks.append(current_block)
    
    return good_blocks

In [ ]:
def smooth_keypoints(block, window_size=3):
    """
    Apply rolling average smoothing to keypoints in a block.
    
    :param block: A list of frames, each frame is a dictionary with 'first_detection_keypoints'.
    :param window_size: Size of the rolling window for averaging.
    :return: A new block with smoothed keypoints.
    """
    # Convert block to numpy array for easier manipulation
    keypoints_array = np.array([frame['first_detection_keypoints'] for frame in block])
    num_frames, num_keypoints, _ = keypoints_array.shape
    
    # Initialize smoothed keypoints array
    smoothed_keypoints = np.copy(keypoints_array)
    
    # Apply rolling average
    for i in range(num_frames):
        start = max(0, i - window_size // 2)
        end = min(num_frames, i + window_size // 2 + 1)
        smoothed_keypoints[i] = np.mean(keypoints_array[start:end], axis=0)
    
    # Update block with smoothed keypoints
    smoothed_block = []
    for i, frame in enumerate(block):
        smoothed_frame = frame.copy()
        smoothed_frame['first_detection_keypoints'] = smoothed_keypoints[i].tolist()
        smoothed_block.append(smoothed_frame)
    
    return smoothed_block

In [ ]:
def calculate_keypoint_displacements(block):
    """
    Calculate the total displacement for each keypoint in a block.
    
    :param block: List of frames, each frame is a dictionary with 'first_detection_keypoints'.
    :return: A list with the total displacement for each keypoint.
    """
    # Assuming each frame's keypoints are in the same order.
    displacements = [0] * len(block[0]['first_detection_keypoints'])  # Initialize displacements
    
    for i in range(1, len(block)):
        prev_keypoints = np.array(block[i-1]['first_detection_keypoints'])
        curr_keypoints = np.array(block[i]['first_detection_keypoints'])
        distances = np.linalg.norm(curr_keypoints - prev_keypoints, axis=1)
        displacements += distances  # Update total displacement for each keypoint

    displacements = np.array(displacements) / len(block)  # Normalize by number of frames
    
    return displacements

def filter_blocks_by_displacement(blocks, threshold):
    """
    Filter blocks to keep those where at least one keypoint's total displacement exceeds the threshold.
    
    :param blocks: List of blocks, each block is a list of frames.
    :param threshold: Displacement threshold for filtering.
    :return: Filtered list of blocks.
    """
    filtered_blocks = []
    
    for block in blocks:
        displacements = calculate_keypoint_displacements(block)
        if np.mean(displacements) > threshold:
            filtered_blocks.append(block)
    
    return filtered_blocks

def get_orig_video_info(file):
    file_path = file  # change to your own video path

    try:
        vid = cv2.VideoCapture(file_path)
        height = vid.get(cv2.CAP_PROP_FRAME_HEIGHT)
        width = vid.get(cv2.CAP_PROP_FRAME_WIDTH)
        fps = vid.get(cv2.CAP_PROP_FPS)

        center_x = (width) / 2
        center_y = (height) / 2
    except cv2.error as e:
        print(f"Caugt cv2 error, setting dummy params")
        width = 0
        height = 0
        center_x = 0
        center_y = 0
        fps = 0
        
    return width, height, center_x, center_y, fps


def find_file_by_basename(directory, base_name):
    """
    Find a file in the specified directory that has the given base name with any extension.

    Args:
    - directory (str): The directory to search in.
    - base_name (str): The base name of the file to find.

    Returns:
    - str: The path of the first matching file found, or None if no match is found.
    """

    for filename in os.listdir(directory):
        if os.path.splitext(filename)[0].lower() == base_name.lower():
            return os.path.join(directory, filename)
    return None

In [ ]:
def reorder_keypoints(keypoints, confidence_scores):
    """
    Reorder the keypoints to the OpenPose format.
    The OpenPose format is as follows:
    0-17: [nose, neck, right_shoulder, right_elbow, right_wrist, left_shoulder, left_elbow, left_wrist,
           right_hip, right_knee, right_ankle, left_hip, left_knee, left_ankle, right_eye, left_eye, right_ear, left_ear]
    The input 'keypoints' is a list of (x, y, c) tuples, where c is the confidence score.
    """

    # Reorder the keypoints to the OpenPose format
    keypoints = [keypoints[i] for i in [0, 17, 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]]
    confidence_scores = [confidence_scores[i] for i in [0, 17, 6, 8, 10, 5, 7, 9, 12, 14, 16, 11, 13, 15, 2, 1, 4, 3]]

    return keypoints, confidence_scores

def rescale_keypoints(keypoints, scale):
    """
    Rescale the keypoints by the given scale.
    The input 'keypoints' is a list of (x, y) tuples
    """

    # Rescale the keypoints
    keypoints = [(x * scale, y * scale) for (x, y) in keypoints]

    return keypoints


def convert_coco_to_openpose(coco_keypoints, confidence_scores):
    """
    Convert COCO keypoints to OpenPose keypoints with the neck keypoint as the midpoint between the two shoulders.
    COCO keypoints format (17 keypoints): [nose, left_eye, right_eye, left_ear, right_ear,
                                           left_shoulder, right_shoulder, left_elbow, right_elbow,
                                           left_wrist, right_wrist, left_hip, right_hip,
                                           left_knee, right_knee, left_ankle, right_ankle]
    OpenPose keypoints format (18 keypoints): COCO keypoints + [neck]
    The neck is not a part of COCO keypoints and is computed as the midpoint between the left and right shoulders.
    """

    # Assuming coco_keypoints is a list of (x, y) tuples
    nose, left_eye, right_eye, left_ear, right_ear, \
    left_shoulder, right_shoulder, left_elbow, right_elbow, \
    left_wrist, right_wrist, left_hip, right_hip, \
    left_knee, right_knee, left_ankle, right_ankle = coco_keypoints

    # Calculate the neck as the midpoint between left_shoulder and right_shoulder
    neck_x = (left_shoulder[0] + right_shoulder[0]) / 2
    neck_y = (left_shoulder[1] + right_shoulder[1]) / 2
    neck = (neck_x, neck_y)


    # Assuming coco_keypoints is a list of (x, y) tuples
    c_nose, c_left_eye, c_right_eye, c_left_ear, c_right_ear, \
    c_left_shoulder, c_right_shoulder, c_left_elbow, c_right_elbow, \
    c_left_wrist, c_right_wrist, c_left_hip, c_right_hip, \
    c_left_knee, c_right_knee, c_left_ankle, c_right_ankle = confidence_scores

    # Calculate the neck as the midpoint between left_shoulder and right_shoulder
    c_neck = (c_left_shoulder + c_right_shoulder) / 2

    # Construct the OpenPose keypoints including the neck
    openpose_keypoints = [
        nose, left_eye, right_eye, left_ear, right_ear,
        left_shoulder, right_shoulder, left_elbow, right_elbow,
        left_wrist, right_wrist, left_hip, right_hip,
        left_knee, right_knee, left_ankle, right_ankle,
        neck  # Adding the neck as the last keypoint
    ]
    
    openpose_confidences = [
        c_nose, c_left_eye, c_right_eye, c_left_ear, c_right_ear,
        c_left_shoulder, c_right_shoulder, c_left_elbow, c_right_elbow,
        c_left_wrist, c_right_wrist, c_left_hip, c_right_hip,
        c_left_knee, c_right_knee, c_left_ankle, c_right_ankle,
        c_neck  # Adding the neck as the last keypoint
    ]

    openpose_keypoints, confidences = reorder_keypoints(openpose_keypoints, openpose_confidences)
    openpose_keypoints = rescale_keypoints(openpose_keypoints, 1)

    return openpose_keypoints, confidences